# Logistic Regression

We will discuss:

- How MLE fits in when it comes to estimating coefficients
- Log-odds
- Odds ratios
- and assumptions regarding MLE


In [ ]:
import pandas as pd
import numpy as np
from main import preprocess_df
from collections import defaultdict as dd


my_df = pd.read_csv(
    "./datasets/lc_data_2007_to_2018.csv",
    low_memory=False,
    encoding="latin1",
    nrows=100000,  # only looking at 100k rows right now for performance
)
pd.set_option("display.max_columns", None)
cleaned_df = preprocess_df(my_df)

Logistic regression is when we calculate a score for how likely something is to happen, via a linear equation. In our case, we want to predict the probability of default from all the factors contained in the dataset. But since we're using a linear equation, this score can range from $-\infin$ to $\infin$, and we could end up with a value that makes no sense probabilistically, like $1.2$. To account for this, we put it through the sigmoid function:

##### $S(z) = \frac{1}{1+e^{-z}}$

This function takes in values from $-\infin$ to $\infin$ and outputs only values between 0 and 1, perfect for probability.


### An Explanation of Why Coefficients are Estimated Using Maximum Likelihood

##### Background

In logistic regression, a function $S(z)$ is created to predict the probability of a binary output from inputs ($x_1, x_2, ...x_n$). $z$ is a linear expression, formed from a linear combination of terms $\beta_nx_n$. Each term is the product of a factor's value ($x_n$) and the weight ($\beta_n$) associated with that factor. 'Coefficients' here refer to the $\beta\text{s}$. The logistic regression system takes all the training data in, and tries to minimise the error between what it predicts vs what the true outcome.

##### Explanation

Betas are estimated with MLE as opposed to Least Squares because of this: the 'true' output value will always be one of yes or no, 1 or 0. Errors are defined as $\epsilon = y-\hat y$. An assumption inherent in Least Squares is that the errors are Gaussian around the prediction. But for any one prediction $\hat y$, the error's two possible values are $-\hat y$ (if $y=0$) or $1-\hat y$ (if $y=1$). Thus the error follows a Bernoulli distribution, not a Gaussian one. Also, Gaussian tails are infinite, so that would imply there is a nonzero probability for the true value to lie outside $[0,1]$. Thus we cannot use Least Squares. The outcomes are Bernoulli, and the way the $\beta\text{s}$ are found is by using MLE to find which $\beta$ values maximise the likelihood of observing what we just saw with the true values.

Essentially, the MLE process goes like this: the model receives data as input, then asks this question: what 'line of best fit' maximises the likelihood of all the data points we observed? The likelihood is the chance that all these events occurred simultaneously, and it's calculated by, of course, multiplying together all the probabilities of each data point being so. But since this looks like, for example, $0.4\cdot 0.8 \cdot 0.7\cdot ...$, over thousands of data points, it quickly becomes a tiny number that computers can't handle because of underflow. So we instead find the log-likelihood, and since the natural log is monotonic, the maximum of the log-likelihood corresponds to the maximum of the likelihood. This turns the multiplication into addition, so the number stays manageable. And then, since the log-likelihood is always negative, the maximum would be the least negative number created from the log-likelihood, and so to turn it into a cost minimisation problem, the sign of the log-likelihood is flipped, so we now want the smallest number, all of which are positive.


Before we start though, there are some NaN values, so we need to do imputation and we have different scales of values, so we need to standardise scaling.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.impute import SimpleImputer

X = cleaned_df.loc[:, cleaned_df.columns != "did_default"]
y = cleaned_df.loc[:, cleaned_df.columns == "did_default"]

y = y.values.ravel()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

imputer = SimpleImputer(strategy="mean")
imputed_X_train = imputer.fit_transform(X_train)
imputed_X_test = imputer.transform(X_test)

scaler = StandardScaler()

scaled_X_train = scaler.fit_transform(imputed_X_train)
scaled_X_test = scaler.transform(imputed_X_test)

Now running the logistic regression itself with our two weights:


In [ ]:
weights = [None, "balanced"]
for w in weights:
    log_reg = LogisticRegression(class_weight=w, random_state=42)
    log_reg.fit(scaled_X_train, y_train)
    y_pred = log_reg.predict(scaled_X_test)
    y_pred_proba = log_reg.predict_proba(scaled_X_test)

    accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    cm = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(cm)

    cr = classification_report(y_test, y_pred, target_names=["Paid", "Defaulted"])
    print("Classification Report:")
    print(cr)
    print("-" * 60)

### Background on Class Weighting

In the test set with seed 42, there are 14028 non-defaulting customers and 3551 defaulting customers. Because the defaulting customers make up a smaller proportion of the population, the model weights them as unimportant and prefers to make sure it gets all the paid customers right. But in reality, it incurs much greater losses to grant a loan to someone who won't pay it back than it is to miss out on interest by refusing a loan to someone who would've paid it back. So we need the model to weight the defaulted cases higher than it currently is.

### Results Without Class Weighting

```
Accuracy: 0.80

Confusion Matrix:
  13615   413
   3045   506

Classification Report:
              precision    recall  f1-score   support
        Paid       0.82      0.97      0.89     14028
   Defaulted       0.55      0.14      0.23      3551
    accuracy                           0.80     17579
   macro avg       0.68      0.56      0.56     17579
weighted avg       0.76      0.80      0.75     17579
```

As can be seen here without class weighting, accuracy is 0.80 (high) but recall on defaulted loans is 0.14 (very low). That means that overall the model is 80% accurate with its predictions of whether a loan is going to be defaulted on or not, but for loans that did default, it correctly predicted 'default' only 14% of the time. That means that of the 3551 loans that did end up defaulting, it predicted full repayment for 3045 of them, which would be disastrous for the bank. Losing the full principal from an approved borrower when she defaults is a much worse outcome than missing out on the interest from rejecting a borrower that would've repaid in full. The reason accuracy was so high here is because the model was predicting 'paid' for 95% of loans, and 80% of loans were indeed paid, so it got a high accuracy score simply because there were more paid loans than defaulted ones.

### Results With Class Weighting

```
Accuracy: 0.68

Confusion Matrix:
  9765 4263
  1302 2249
Classification Report:
              precision    recall  f1-score   support
        Paid       0.88      0.70      0.78     14028
   Defaulted       0.35      0.63      0.45      3551
    accuracy                           0.68     17579
   macro avg       0.61      0.66      0.61     17579
weighted avg       0.77      0.68      0.71     17579
```

When we introduce class weighting, we can see that accuracy drops from 0.80 to 0.68. This is because overall it is more conservative, meaning it predicts default for more loans, including many that actually were repaid. This is a worthy sacrifice though, because the number of defaulted loans it predicted as 'paid' (false negatives) dropped from 3045 to 1302, a huge relative difference. The model went from catching 14% of defaulters to catching 63% of them. The reason for this change is because we changed the way the model weights errors. Instead of making each loan equal weight, which is what caused the model to just guess 'paid' almost every time and get an 0.80 accuracy score but fail to flag a lot of loans that defaulted, the approach used here was to make each population of loans equally weighted. Therefore, since there are fewer defaulted loans, each one individually is weighted higher than an individual paid loan. So the error of predicting 'paid' for a loan that actually defaulted hits harder than predicting 'defaulted' for a loan that was actually repaid, which is the way it should be.  
However, that being said, precision did take a hit with the more conservative approach, dropping from 0.55 to 0.35. This means of all the people predicted to default, the proportion of them who actually did dropped from 0.55 to 0.35. Therefore, when we balance class weights, 65% of people who we predicted to default actually would've repaid. This is a lot of interest to miss out on, but the bank must accept that missing out on some interest is worth the cost of avoiding the loss of entire principals.


---

Now we will look at what the model's doing under the hood. We will first look at the coefficients used by the model (the weights) for each of the factors like annual income, DTI etc.


In [ ]:
weights = [None, "balanced"]
for w in weights:
    print(f"\nThis is for weight: {w}")
    log_reg = LogisticRegression(class_weight=w, random_state=42)
    log_reg.fit(scaled_X_train, y_train)
    rounded_coeffs = [round(c, 3) for c in log_reg.coef_[0]]
    coeffs = pd.DataFrame(
        {
            "Feature": X.columns,
            "Coefficient": rounded_coeffs,
        }
    )
    coeffs = coeffs.sort_values(by="Coefficient", ascending=False, key=abs)
    print(coeffs)

For balanced classes, the coefficients with the highest absolute value, and therefore with the most influence on the prediction, are:

1. Interest rate (0.475)
2. Term (0.385)
3. Installment (0.318)

For non-balanced classes, the top 3 coefficients were the same but with slightly different values.

##### Log-odds Background

The probability of something happening is fixed between 0 and 1. Say we want to use a computer to predict the probability of something, like whether a loan will default. But because we're giving a computer data points and asking it to predict based on that, it wants to use a straight line (linear equation) to make predictions. This straight line would naively have a feature on the x-axis and the probability of default on the y-axis. Because the line's straight, if you keep going left or right, it will eventually rise over 100% or fall below 0%. For example, if higher loan amounts were an indicator of higher default probability, then if you kept going to the right, eventually default probability would eclipse 100%, which isn't possible.  
So to fix this, we must change our unit to something other than probability to allow us to span the whole number line, just like how a linear equation does. We need to transform the limited probability scale ($[0,1]$) to a scale that spans the whole number line with a new unit, and then transform it later ('squash it down') to be between 0 and 1, which can be done with the sigmoid function.  
Our first move is to switch to odds instead of probability. Odds is the ratio of the chance of something happening to the chance it doesn't. So if p is the chance of something happening, then $Odds = \frac{p}{1-p}$. Now our unit has no ceiling, but it still can't be less than 0. So we now take the natural log of the odds: since odds lies within $[0, \infin]$, log-odds lies within $[-\infin, \infin]$. We have successfully covered the whole number line.  
So our linear equation's units are in log-odds. If the linear regression process determines feature X to have a slope of m, then it means that a rise of 1 unit of X causes a rise of $m$ in log-odds, which means a rise of $e^m$ in odds. 1 unit of X here means 1 standard deviation of X, since we've scaled values using scikit-learn's StandardScaler.  
So in summary, we use log-odds because it opens up the whole number line to us, and then we transform that log-odds value to odds or probability as we please.

##### A mathematical explanation of transforming log-odds to probability (and derivation of the sigmoid function)

log-odds = $z$  
$\therefore Odds = e^z$  
$Odds = \frac{p}{1-p}$  
$\therefore Odds \cdot (1-p) = p$  
$\therefore Odds - (Odds\cdot p) = p$  
$\therefore Odds = p + (Odds\cdot p)$  
$\therefore Odds = p\cdot (1+Odds)$  
$\therefore p = \frac{Odds}{1+Odds}$  
$\therefore p = \frac{e^z}{1+e^z}=\frac{1}{1+e^{-x}}$

##### Log-odds interpretation

Since upon transformation from log-odds to odds we exponentiate, for a line with gradient $m$, it works out that odds are multiplied by a factor of $e^m$ when the factor on the x-axis rises by 1 standard deviation. Since each coefficient is the gradient m, we can say that a rise of 1 standard deviation in a factor X multiplies the odds of default by $e^{coefficient}$.  
For example, the coefficient of interest rate was 0.475. This means that when interest rate rises by 1 standard deviation from the mean, the log-odds rise by 0.475, and the odds of default are multiplied by $e^{0.475}=1.608$. This means if we go 2 standard deviations, the odds of default are multiplied by $1.608^2=2.586$.
For term, the coefficient was 0.385, and so the log-odds rise by 0.385, and the odds of default rise by $e^{0.385}=1.470$ for each additional standard deviation.


### Assumptions Inherent to MLE

When using MLE, we make a few assumptions, including:

- Data points are i.i.d. Independence is necessary because we calculate likelihood by multiplying together each event's probability.
  - Plausibility: likely. Independence and identical distribution are likely here because someone defaulting/paying back a loan is unlikely to affect others. However, it is possible that some act of God like a recession would influence lots of people to default, and then others would follow suit, thus violating independence. But this is unlikely.
- The data follows a Bernoulli distribution. This is necessary because we're minimising the error based on the data only being able to take values 0 or 1.
  - Plausibility: certain. It's definitionally true that the outcome variable is binary, as the only two options are 'Defaulted' and 'Fully Paid.'
- The relationship between the factors and the log-odds (which is then transfromed to probability) of default is linear.
  - Plausibility: debatable. It's not obvious whether, say, probability of default explodes once you get below $30k annual income or say, shrinks to nothing once the loan amount goes below $5k.
- The sample size is large to reduce the effects of outliers and produce an unbiased estimate.
  - Plausibility: certain. N > 17000, which is very large.


## A Discussion on the Dataset's Selection Bias

The dataset used in this project contains only loans that were approved by LendingClub. This means that any predictive probabilities produced by the logistic regression model are interpreted as on the condition that they were approved. The model is estimating $P(Default | X\cap Approved)$, rather than the true population probability $P(Default | X)$.  
Furthermore, LendingClub's approval filter may distort or obscure the relationships between a factor and the outcome. For example, it's possible LC rejects any applicant with a FICO score below a certain threshold, and so this may result in underestimating the effect of FICO score on probability of rejection, since we're only seeing people with decent FICO scores. To explain further: let's say that in the dataset we have access to, FICO score carries only a moderate weight. This is very plausible since to get approved by LC, you must at least have a decent FICO score, so there won't be a huge variety of scores, and so it's not much of an indicator of default. But if you're using this model to decide whether to approve the loan of some applicant, and they have a very low FICO score, but it's only weighted moderately in the decision, then they might get approved when they should not have.  
Lastly, coefficients cannot be treated as population-wide truths, since as mentioned above, different factors may carry different weights in different subsets of the population. For example, in the subset of the population with decent FICO scores (which the dataset is likely to be), differences in FICO score become less important as the variance is lower than is natural, and perhaps in this subset of the population DTI becomes more important as an indicator of default, even though in the general population it makes only a marginal difference. And in the general population, where we haven't filtered for good FICO scores, FICO score may be a stronger indicator.
